In [1]:
import copy
import utils
import torch
import numpy as np
import matplotlib.pyplot as plt

from env import Env
from agent import Agent
from torchsummary import summary

# Initialise Environment

In [2]:
#initialise environment
min_x, max_x =  -0.110 - 0.175,   -0.110 + 0.175
min_y, max_y =   0.535 - 0.175,    0.535 + 0.175
min_z, max_z =               0,              0.4 

workspace_lim = np.asarray([[min_x, max_x], 
                            [min_y, max_y],
                            [min_z, max_z]])

print(f"workspace space: \n{workspace_lim}")

obj_dir = 'objects/blocks/'
N_obj   = 3

env = Env(obj_dir, N_obj, workspace_lim)

workspace space: 
[[-0.285  0.065]
 [ 0.36   0.71 ]
 [ 0.     0.4  ]]
[SUCCESS] restart environment
[setup_rgbd_cam] 
 [[ 9.99999992e-01  2.89813737e-05  1.26784815e-04 -1.06169154e-01]
 [ 2.90796779e-05 -9.99999699e-01 -7.75429384e-04  5.35611757e-01]
 [ 1.26762304e-04  7.75433064e-04 -9.99999691e-01  3.42376167e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[SUCCESS] setup rgbd camera
[SUCCESS] load obj paths
[SUCCESS] randomly choose objects
[SUCCESS] randomly choose object colors
object 0: shape_0, pose: [-0.13458401716013738, 0.5383885981107812, 0.15, 1.4699236738507695, 3.2385404117966625, 0.8766803191757007]
object 1: shape_1, pose: [-0.1463967293907642, 0.6587041825104886, 0.15, 4.4174883030573895, 3.8604746730793336, 5.157519176943474]
object 2: shape_2, pose: [-0.2463409058743293, 0.5088075607723527, 0.15, 3.5104077297252236, 2.114596282370982, 0.3670739209043347]
[SUCCESS] add objects to simulation


# Test Environment Reset

In [3]:
env.reset(reset_obj = False)

[SUCCESS] restart environment
[setup_rgbd_cam] 
 [[ 9.99999990e-01  3.01665057e-05  1.38725731e-04 -1.06171375e-01]
 [ 3.02703689e-05 -9.99999719e-01 -7.48753515e-04  5.35606429e-01]
 [ 1.38703105e-04  7.48757707e-04 -9.99999710e-01  3.42390860e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[SUCCESS] setup rgbd camera
[SUCCESS] load obj paths
object 0: shape_0, pose: [-0.12271089653634884, 0.5810109787367578, 0.018970500570462914, -2.931622389679219, -0.7626821183150228, -2.842469342182401]
object 1: shape_1, pose: [-0.15086120418465288, 0.6328634692767564, 0.015098536641870778, 2.720410231954403, -0.3019349380592663, 2.1567981351597045]
object 2: shape_2, pose: [-0.22501116827563156, 0.498512616991673, 0.03458481677287044, 1.5750400880280504, 1.3624525601345234, 2.518891099015809]
[SUCCESS] add objects to simulation


# Initialise Agent

In [4]:
agent = Agent(env, N_batch = 8)

[SUCCESS] initialise environment
[SUCCESS] initialise networks
[SUCCESS] initialise memory buffer


/home/ryan/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969824/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Check Guidance

In [5]:
agent.is_debug = True
delta_move = agent.env.push_guidance_generation(max_move = 0.05)

gripper_pos = np.array([-0.11122626281611381, 0.4855598136140757, 0.2684023847637833])
for i in range(len(delta_move)):
    gripper_pos += np.array(delta_move[i][0:3])
    print(f'gripper_pos: {gripper_pos}, type: {delta_move[i][-1]}')

[return_home] [-0.0006938476395508227, -0.00010905491541380616, 2.827573549383989e-05] [0, 0, 0]


In [ ]:
agent.interact_by_guidance(max_episode = 1)

# Check Encoder

In [ ]:
summary(agent.encoder, input_size=(1, 128, 128))

In [ ]:
#get color
color_img, depth_img = agent.env.get_rgbd_data()
print(f'dmin: {np.min(depth_img[:])}, dmax: {np.max(depth_img[:])}')

#preprocess data
in_color_img, in_depth_img = agent.preprocess_input(color_img, depth_img)
print(in_color_img.shape)
print(in_depth_img.shape)

#add the extra dimension in the 1st dimension
in_color_img = in_color_img.unsqueeze(0)
in_depth_img = in_depth_img.unsqueeze(0)
print(in_color_img.shape)
print(in_depth_img.shape)

#feed into encoder
with torch.no_grad():
    latent_vector, reconstructed = agent.encoder(in_depth_img)

print(f'dmin: {torch.min(in_depth_img)}, dmax: {torch.max(in_depth_img)}')
print('latent vector shape: ', latent_vector.shape)
print('reconstructed shape: ', reconstructed.shape)

#show depth image
plt.imshow(in_depth_img[0].permute(1,2,0))

# Check Actor

In [ ]:
#feed into actor
with torch.no_grad():
    a, a_type, z, normal, a_type_probs = agent.actor.get_actions(latent_vector)
    
print(f"action: {a}, action_type: {a_type}")

# Check Critic

In [ ]:
#feed into actor
with torch.no_grad():

    #compute one hot vector
    a_type_onehot = torch.nn.functional.one_hot(a_type.long(), num_classes = 3).float()

    q1 = agent.critic1(state = latent_vector, action = a, action_type = a_type_onehot)
    q2 = agent.critic2(state = latent_vector, action = a, action_type = a_type_onehot)

    tq1 = agent.critic1_target(state = latent_vector, action = a, action_type = a_type_onehot)
    tq2 = agent.critic2_target(state = latent_vector, action = a, action_type = a_type_onehot)

print(f"q1: {q1}, q2: {q2}, tq1: {tq1}, tq2: {tq2}")

# Testing raw data and preprocess input

In [ ]:
color_img, depth_img = agent.env.get_rgbd_data()

In [ ]:
in_color_img, in_depth_img = agent.preprocess_input(color_img, depth_img)

In [ ]:
print(in_color_img.shape)
print(in_depth_img.shape)

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(in_depth_img.permute((1,2,0)))
ax[1].imshow(in_color_img.permute((1,2,0)))
plt.show()

# Test interact

In [ ]:
agent.interact()